#TWEETS DISASTER

Spark session

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

install necessaire sur le cluster: 

my cluster > config > librairies > new lib > pyPI > sparl-nlp > install
my cluster > config > librairies > new lib > maven > coordinaes > com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.3 > install



In [0]:
%pip install spark-nlp
%pip install openpyxl




Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


Import necessaire

chargement du dataframe

In [0]:
dbutils.fs.ls("/FileStore/tables")


Out[2]: [FileInfo(path='dbfs:/FileStore/tables/test.csv', name='test.csv', size=420783, modificationTime=1730799920000),
 FileInfo(path='dbfs:/FileStore/tables/train-1.csv', name='train-1.csv', size=987712, modificationTime=1730380438000),
 FileInfo(path='dbfs:/FileStore/tables/train-2.csv', name='train-2.csv', size=987712, modificationTime=1730795566000),
 FileInfo(path='dbfs:/FileStore/tables/train-3.csv', name='train-3.csv', size=987712, modificationTime=1730795608000),
 FileInfo(path='dbfs:/FileStore/tables/train.csv', name='train.csv', size=987712, modificationTime=1730365543000)]

In [0]:
from pyspark.sql.functions import lit

df_train = spark.read.format("csv").option("header", True) \
                     .load("/FileStore/tables/train.csv")

df_train.show(20)


+---+-------+--------+--------------------+------+
| id|keyword|location|                text|target|
+---+-------+--------+--------------------+------+
|  1|   null|    null|Our Deeds are the...|     1|
|  4|   null|    null|Forest fire near ...|     1|
|  5|   null|    null|All residents ask...|     1|
|  6|   null|    null|13,000 people rec...|     1|
|  7|   null|    null|Just got sent thi...|     1|
|  8|   null|    null|#RockyFire Update...|     1|
| 10|   null|    null|#flood #disaster ...|     1|
| 13|   null|    null|I'm on top of the...|     1|
| 14|   null|    null|There's an emerge...|     1|
| 15|   null|    null|I'm afraid that t...|     1|
| 16|   null|    null|Three people died...|     1|
| 17|   null|    null|Haha South Tampa ...|     1|
| 18|   null|    null|#raining #floodin...|     1|
| 19|   null|    null|#Flood in Bago My...|     1|
| 20|   null|    null|Damage to school ...|     1|
| 23|   null|    null|      What's up man?|     0|
| 24|   null|    null|       I 

In [0]:
%pip freeze

argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
attrs==21.4.0
backcall==0.2.0
backports.entry-points-selectable==1.2.0
beautifulsoup4==4.11.1
black==22.3.0
bleach==4.1.0
boto3==1.21.32
botocore==1.24.32
certifi==2021.10.8
cffi==1.15.0
chardet==4.0.0
charset-normalizer==2.0.4
click==8.0.4
cryptography==3.4.8
cycler==0.11.0
Cython==0.29.28
dbus-python==1.2.16
debugpy==1.5.1
decorator==5.1.1
defusedxml==0.7.1
distlib==0.3.6
distro==1.4.0
distro-info==0.23+ubuntu1.1
docstring-to-markdown==0.11
entrypoints==0.4
et_xmlfile==2.0.0
executing==0.8.3
facets-overview==1.0.0
fastjsonschema==2.16.2
filelock==3.9.0
fonttools==4.25.0
idna==3.3
ipykernel==6.15.3
ipython==8.5.0
ipython-genutils==0.2.0
ipywidgets==7.7.2
jedi==0.18.1
Jinja2==2.11.3
jmespath==0.10.0
joblib==1.1.1
jsonschema==4.4.0
jupyter-client==6.1.12
jupyter_core==4.11.2
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.0
kiwisolver==1.3.2
MarkupSafe==2.0.1
matplotlib==3.5.1
matplotlib-inline==0.1.2
mccabe==0.7.0
mis

Trie, pré-traitement

In [0]:
from pyspark.sql import Window
from pyspark.sql import functions as F


# fonctions de tri 

def replace_text(df):
    text_columns = ["id", "keyword", "location"]

 
    new_text_value = F.coalesce(*[F.when(F.col(c).cast("string").isNotNull() & (F.col(c) != ""), F.col(c)) for c in text_columns])

    df = (df
          .withColumn("text", F.when(F.col("text").isNull(), new_text_value).otherwise(F.col("text")))
          .withColumn("id", F.when((F.col("text") == F.col("id")) & F.col("id").cast("string").isNotNull(), None).otherwise(F.col("id")))
          .withColumn("keyword", F.when((F.col("text") == F.col("keyword")), None).otherwise(F.col("keyword")))
          .withColumn("location", F.when((F.col("text") == F.col("location")), None).otherwise(F.col("location")))
         )
    return df



def replace_target(df):
    
    int_columns = ["keyword", "location", "text"]

    
    new_target_value = F.coalesce(*[F.when((F.col(c) == 0) | (F.col(c) == 1), F.col(c)) for c in int_columns])

    df = (df
          .withColumn("target", F.when(F.col("target").isNull(), new_target_value).otherwise(F.col("target")))
          .withColumn("keyword", F.when((F.col("target") == F.col("keyword")), None).otherwise(F.col("keyword")))
          .withColumn("location", F.when((F.col("target") == F.col("location")), None).otherwise(F.col("location")))
          .withColumn("text", F.when((F.col("target") == F.col("text")), None).otherwise(F.col("text"))))

    return df

def replace_location(df):
    df = df.withColumn("location", F.when(F.col("location").contains("?"), None).otherwise(F.col("location")))
    return df

def drop_null_target(df):
    df = df.filter(F.col("target").isNotNull())
    return df
    
def drop_null_text(df):
    df = df.filter(F.col("text").isNotNull())
    return df



def reset_ids(df):
    
    window_spec = Window.orderBy(F.lit(1))
    df = df.withColumn("id", F.row_number().over(window_spec))
    return df


In [0]:
# on verifie
df2=replace_text(df_train)
null_count = df2.select(F.sum(F.col("target").isNull().cast("int"))).alias("null_count")
null_count.show()
df3=replace_target(df2)
null_count = df3.select(F.sum(F.col("target").isNull().cast("int"))).alias("null_count")
null_count.show()

df3.show(100)

+----------------------------------+
|sum(CAST((target IS NULL) AS INT))|
+----------------------------------+
|                              1211|
+----------------------------------+

+----------------------------------+
|sum(CAST((target IS NULL) AS INT))|
+----------------------------------+
|                               775|
+----------------------------------+

+----+--------+--------------------+--------------------+------+
|  id| keyword|            location|                text|target|
+----+--------+--------------------+--------------------+------+
|   1|    null|                null|Our Deeds are the...|     1|
|   4|    null|                null|Forest fire near ...|     1|
|   5|    null|                null|All residents ask...|     1|
|   6|    null|                null|13,000 people rec...|     1|
|   7|    null|                null|Just got sent thi...|     1|
|   8|    null|                null|#RockyFire Update...|     1|
|  10|    null|                null|#flood 

In [0]:
def tri(df):
    df = replace_text(df)  # remplace le text null, si il existe dans une des autres colonnes
    df = replace_target(df) # remplace le target null si il existe dans une autre colonne
    df = replace_location(df)  #supprime le contenue de location si il y'à au moin un "?"
    df = drop_null_target(df) # supprime les target qui n'ont pas été remplacé car manquant
    df = drop_null_text(df)   # supprime le text devenue nulle , si le target était à l'intérieur
    df = reset_ids(df)  # Réinitialisation des IDs correctement
    return df

In [0]:
data = tri(df_train)
data.show(400)

+---+-------------------+--------------------+--------------------+------+
| id|            keyword|            location|                text|target|
+---+-------------------+--------------------+--------------------+------+
|  1|               null|                null|Our Deeds are the...|     1|
|  2|               null|                null|Forest fire near ...|     1|
|  3|               null|                null|All residents ask...|     1|
|  4|               null|                null|13,000 people rec...|     1|
|  5|               null|                null|Just got sent thi...|     1|
|  6|               null|                null|#RockyFire Update...|     1|
|  7|               null|                null|#flood #disaster ...|     1|
|  8|               null|                null|I'm on top of the...|     1|
|  9|               null|                null|There's an emerge...|     1|
| 10|               null|                null|I'm afraid that t...|     1|
| 11|               null|

ETUDE 1: prise en compte uniquement du text comme variable explicative (location et keyword laisser de coté)

In [0]:

data1= data.select("id","text","target")
data.count()


Out[6]: 7611

In [0]:
data1 = data1.withColumn("target", F.col("target").cast("int"))
data1 = data1.withColumnRenamed("target", "label")
data1.show()

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Our Deeds are the...|    1|
|  2|Forest fire near ...|    1|
|  3|All residents ask...|    1|
|  4|13,000 people rec...|    1|
|  5|Just got sent thi...|    1|
|  6|#RockyFire Update...|    1|
|  7|#flood #disaster ...|    1|
|  8|I'm on top of the...|    1|
|  9|There's an emerge...|    1|
| 10|I'm afraid that t...|    1|
| 11|Three people died...|    1|
| 12|Haha South Tampa ...|    1|
| 13|#raining #floodin...|    1|
| 14|#Flood in Bago My...|    1|
| 15|Damage to school ...|    1|
| 16|      What's up man?|    0|
| 17|       I love fruits|    0|
| 18|    Summer is lovely|    0|
| 19|   My car is so fast|    0|
| 20|What a goooooooaa...|    0|
+---+--------------------+-----+
only showing top 20 rows



In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *



documentAssembler = DocumentAssembler() \
                      .setCleanupMode("inplace") \
                      .setInputCol("text") \
                      .setOutputCol("document")

documents = documentAssembler.transform(data1)
documents.show(12, 100)

+---+----------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------+
| id|                                                                                                text|label|                                                                                            document|
+---+----------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------+
|  1|                               Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all|    1|[{document, 0, 68, Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all, {senten...|
|  2|                                                              Forest fire near La Ronge Sask. Canada|    1|                    [{document, 

In [0]:
tokenizer = Tokenizer() \
                  .setInputCols(["document"]) \
                  .setOutputCol("token")

tokensEch = tokenizer.fit(documents).transform(documents)
tokensEch.select("token").show(1, 300)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                                                                                                                       token|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{token, 0, 2, Our, {sentence -> 0}, []}, {token, 4, 8, Deeds, {sentence -> 0}, []}, {toke

In [0]:

embeddings = WordEmbeddingsModel.pretrained("glove_6B_300", "xx") \
                                .setInputCols("document", "token") \
                                .setOutputCol("glove_embeddings")


embedsEch = embeddings.transform(tokensEch)
embedsEch.select("glove_embeddings").show(5, 200)

glove_6B_300 download started this may take some time.
Approximate size to download 426.2 MB
[OK!]
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                        glove_embeddings|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{word_embeddings, 0, 2, Our, {isOOV -> false, pieceId -> -1, isWordStart -> true, token -> Our, sentence -> 0}, [-0.025179, 0.07837, -0.17637, -0.12285, -0.13156, 0.26591, -0.60064, 0.46653, 0.042...|
|[{word_embeddings, 0, 5, Forest, {isOOV -> false, pieceId -> -1, isWordStart -> true, to

In [0]:
embeddingsSentence = SentenceEmbeddings() \
                           .setInputCols(["document", "glove_embeddings"]) \
                           .setOutputCol("sentence_embeddings_glove") \
                           .setPoolingStrategy("AVERAGE")


embeddingsFinisher = EmbeddingsFinisher() \
                           .setInputCols("sentence_embeddings_glove") \
                           .setOutputCols("finished_embeddings_glove") \
                           .setOutputAsVector(True) \
                           .setCleanAnnotations(False)

sentencesEch = embeddingsSentence.transform(embedsEch)
embeddingsFinisher.transform(sentencesEch).select("finished_embeddings_glove").show(5, 200)



+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                               finished_embeddings_glove|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[-0.06928376853466034,-0.01271053310483694,0.00862519908696413,-0.25277167558670044,-0.12423738092184067,0.07902661710977554,-0.07260183990001678,0.152671679854393,0.07146238535642624,-1.411920666...|
|[[-0.05899748578667641,0.054601624608039856,-0.1071816235780716,-0.11128462851047516,-0.03310374915599823,-0.08629648387432098,-0.10258549451828003,0.19005662202835083,-0.0949773639440536

In [0]:
from pyspark.ml import Pipeline
# a resoudre
pipelineGlove = Pipeline() \
                      .setStages([ \
                         documentAssembler, \
                         tokenizer, \
                         embeddings, \
                         embeddingsSentence, \
                         embeddingsFinisher \
                      ])


resultGlove = pipelineGlove.fit(data1).transform(data1).cache()
resultGlove.select("finished_embeddings_glove").show(5, 200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                               finished_embeddings_glove|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[-0.06928376853466034,-0.01271053310483694,0.00862519908696413,-0.25277167558670044,-0.12423738092184067,0.07902661710977554,-0.07260183990001678,0.152671679854393,0.07146238535642624,-1.411920666...|
|[[-0.05899748578667641,0.054601624608039856,-0.1071816235780716,-0.11128462851047516,-0.03310374915599823,-0.08629648387432098,-0.10258549451828003,0.19005662202835083,-0.0949773639440536

In [0]:
resultGlove= embeddingsFinisher.transform(sentencesEch)
resultGlove.show(5,200)

+---+-------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------

In [0]:
allEmbeddings = resultGlove.selectExpr("finished_embeddings_glove[0]", "label")



In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Initialiser un modèle de régression logistique
lr = LogisticRegression(featuresCol="finished_embeddings_glove[0]", labelCol="label")

# Construire une grille de paramètres pour le tuning
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.01, 0.04, 0.1, 0.7, 1.0]).build()

# Configurer CrossValidator pour le tuning
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)

# Entraîner le modèle
cvModel = crossval.fit(allEmbeddings)

# Évaluer le modèle
predictions = cvModel.transform(allEmbeddings)
evaluator = BinaryClassificationEvaluator(labelCol="label")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8593487386818074


In [0]:
predictions.show()

+----------------------------+-----+--------------------+--------------------+----------+
|finished_embeddings_glove[0]|label|       rawPrediction|         probability|prediction|
+----------------------------+-----+--------------------+--------------------+----------+
|        [-0.0692837685346...|    1|[0.9382492194644,...|[0.71874587228663...|       0.0|
|        [-0.0589974857866...|    1|[-2.8436065170257...|[0.05501274697257...|       1.0|
|        [-0.1323560625314...|    1|[-1.2904097054499...|[0.21578347199762...|       1.0|
|        [-0.1698394864797...|    1|[-3.3341182097690...|[0.03441910121366...|       1.0|
|        [-0.2219433784484...|    1|[-0.1177100032644...|[0.47060643025436...|       1.0|
|        [-0.1226526275277...|    1|[-1.6340075086796...|[0.16328211205020...|       1.0|
|        [-0.1124185249209...|    1|[-3.2788737206827...|[0.03630309889191...|       1.0|
|        [-0.1336046308279...|    1|[0.05820328758255...|[0.51454671556217...|       0.0|
|        [

In [0]:

# Calculer les vrais positifs, faux positifs, faux négatifs, et vrais négatifs
tp = predictions.filter((F.col("prediction") == 1.0) & (F.col("label") == 1)).count()
fp = predictions.filter((F.col("prediction") == 1.0) & (F.col("label") == 0)).count()
fn = predictions.filter((F.col("prediction") == 0.0) & (F.col("label") == 1)).count()
tn = predictions.filter((F.col("prediction") == 0.0) & (F.col("label") == 0)).count()

# Fonction pour calculer la F1-Score
def calculate_f1_score(tp, fp, fn):
    # Calcul de la précision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    # Calcul du rappel
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # Calcul de la F1-Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1_score, precision, recall

# Calculer et afficher les résultats
f1, precision, recall = calculate_f1_score(tp, fp, fn)
print(f"Précision : {precision:.3f}")
print(f"Rappel : {recall:.3f}")
print(f"F1-Score : {f1:.3f}")


Précision : 0.808
Rappel : 0.678
F1-Score : 0.737


In [0]:
test = spark.read.format("csv").option("header", True) \
                     .load("/FileStore/tables/test.csv")

                    

In [0]:
def drop_null_id(df):
    df = df.filter(F.col("id").isNotNull())
    return df


def tri_test(df):
    df = replace_text(df)  # remplace le text null, si il existe dans une des autres colonnes
    df = replace_location(df)  #supprime le contenue de location si il y'à au moin un "?"
    df = drop_null_text(df)   # supprime le text devenue nulle , si le target était à l'intérieur
    df = drop_null_id(df)
    return df
    
test = tri_test(test)
test.show(70)

+---+-------------------+--------------------+--------------------+
| id|            keyword|            location|                text|
+---+-------------------+--------------------+--------------------+
|  0|               null|                null|Just happened a t...|
|  2|               null|                null|Heard about #eart...|
|  3|               null|                null|there is a forest...|
|  9|               null|                null|Apocalypse lighti...|
| 11|               null|                null|Typhoon Soudelor ...|
| 12|               null|                null|We're shaking...I...|
| 21|               null|                null|They'd probably s...|
| 22|               null|                null|   Hey! How are you?|
| 27|               null|                null|    What a nice hat?|
| 29|               null|                null|           Fuck off!|
| 30|               null|                null|No I don't like c...|
| 35|               null|                null|NO

In [0]:
test.count()

Out[30]: 3261

In [0]:
#pblm pipeline
resultGloveTest = pipelineGlove.fit(test).transform(test).cache()
resultGloveTest.select("finished_embeddings_glove").show(5, 200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                               finished_embeddings_glove|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[[0.1203618273139,0.037857331335544586,-0.09125516563653946,-0.1840178370475769,-0.12349233031272888,-0.013411663472652435,-0.028739996254444122,0.07825849205255508,0.1439501792192459,-1.3446332216...|
|[[-0.1545984447002411,-0.001895270892418921,0.05120636150240898,-0.10950100421905518,0.04543263465166092,0.08200632780790329,0.044895343482494354,0.10999364405870438,0.12287580221891403,-

In [0]:
emTest = resultGloveTest.selectExpr("finished_embeddings_glove[0]")
resultats_predictions = cvModel.transform(emTest)
resultats_predictions.show()
resultats_predictions.count()


+----------------------------+--------------------+--------------------+----------+
|finished_embeddings_glove[0]|       rawPrediction|         probability|prediction|
+----------------------------+--------------------+--------------------+----------+
|        [0.1203618273139,...|[-3.2802326250087...|[0.03625558731945...|       1.0|
|        [-0.1545984447002...|[0.55933252140506...|[0.63629808455223...|       0.0|
|        [-0.1126485243439...|[-0.7992504646863...|[0.31018587456981...|       1.0|
|        [0.03537800163030...|[1.49828696300559...|[0.81731884379785...|       0.0|
|        [-0.2820979952812...|[-4.9340999596542...|[0.00714550979058...|       1.0|
|        [-0.0528950020670...|[-2.1232957231361...|[0.10685313358493...|       1.0|
|        [-0.1184807270765...|[1.66507061714492...|[0.84091749933997...|       0.0|
|        [-0.1558066755533...|[3.95616673019964...|[0.98122299436241...|       0.0|
|        [-0.1850659996271...|[3.69956826890747...|[0.97586281142469...|    

In [0]:
test.printSchema()
resultats_predictions.printSchema()

root
 |-- id: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- location: string (nullable = true)
 |-- text: string (nullable = true)

root
 |-- finished_embeddings_glove[0]: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [0]:
test = test.withColumn("row_index", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))
resultats_predictions = resultats_predictions.withColumn("row_index", F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))

# Joindre les deux DataFrames sur "row_index"
df_combined = test.join(resultats_predictions, on="row_index", how="inner").drop("row_index")

df_combined.show()

+---+-------+--------------------+--------------------+----------------------------+--------------------+--------------------+----------+
| id|keyword|            location|                text|finished_embeddings_glove[0]|       rawPrediction|         probability|prediction|
+---+-------+--------------------+--------------------+----------------------------+--------------------+--------------------+----------+
|  0|   null|                null|Just happened a t...|        [0.1203618273139,...|[-3.2802326250087...|[0.03625558731945...|       1.0|
|  2|   null|                null|Heard about #eart...|        [-0.1545984447002...|[0.55933252140506...|[0.63629808455223...|       0.0|
|  3|   null|                null|there is a forest...|        [-0.1126485243439...|[-0.7992504646863...|[0.31018587456981...|       1.0|
|  9|   null|                null|Apocalypse lighti...|        [0.03537800163030...|[1.49828696300559...|[0.81731884379785...|       0.0|
| 11|   null|                null|

In [0]:

final = df_combined.select("id", "prediction")
final.show()

+---+----------+
| id|prediction|
+---+----------+
|  0|       1.0|
|  2|       0.0|
|  3|       1.0|
|  9|       0.0|
| 11|       1.0|
| 12|       1.0|
| 21|       0.0|
| 22|       0.0|
| 27|       0.0|
| 29|       0.0|
| 30|       0.0|
| 35|       0.0|
| 42|       0.0|
| 43|       0.0|
| 45|       0.0|
| 46|       1.0|
| 47|       0.0|
| 51|       0.0|
| 58|       0.0|
| 60|       0.0|
+---+----------+
only showing top 20 rows



In [0]:
resultats_pandas = final.toPandas()

# Convertir la colonne 'prediction' en entier
resultats_pandas['prediction'] = resultats_pandas['prediction'].astype(int)

# Créer une nouvelle colonne avec id et prediction séparés par une virgule
resultats_pandas['id,target'] = resultats_pandas['id'] + ',' + resultats_pandas['prediction'].astype(str)

# Conserver uniquement la nouvelle colonne 'id_target' pour le fichier Excel
resultats_pandas_final = resultats_pandas[['id,target']]

# Sauvegarder le DataFrame modifié en tant que fichier Excel
resultats_pandas_final.to_excel("/tmp/predictions.xlsx", index=False)



print("Fichier predictions.xlsx créé avec succès.")

Fichier predictions.xlsx créé avec succès.


In [0]:
resultats_pandas

,id,prediction,"id,target"
0,0,1,"0,1"
1,2,0,"2,0"
2,3,1,"3,1"
3,9,0,"9,0"
4,11,1,"11,1"
...,...,...,...
3256,10861,1,"10861,1"
3257,10865,1,"10865,1"
3258,10868,1,"10868,1"
3259,10874,1,"10874,1"


In [0]:
import base64
from IPython.display import HTML

file_path = "/tmp/predictions.xlsx"

# Lire le fichier en mode binaire et encoder en base64
with open(file_path, "rb") as f:
    data = f.read()
    b64_data = base64.b64encode(data).decode()

# Créer un lien de téléchargement
download_link = f'<a download="predictions.xlsx" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64_data}">Cliquez ici pour télécharger predictions.xlsx</a>'

# Afficher le lien dans le notebook
display(HTML(download_link))

Cliquez ici pour télécharger predictions.xlsx

resultat avec glove : 0,78394
